### 필요한 라이브러리 import

In [1]:
import import_ipynb
from IPython.core.display import HTML
from code3_Analyzing_Similarity import *
from code4_Crawling_Event import *

importing Jupyter notebook from code3_Analyzing_Similarity.ipynb
importing Jupyter notebook from code4_Crawling_Event.ipynb


### 각종 보조 함수
목적: 가독성을 높이고 편하게 여러 번 사용하기 위해
1. setPart: 마지막 글자의 받침 유무에 따라 뒤에 붙는 조사를 반환하는 함수
2. setChar: HTML 문자를 특수문자로 변환하는 함수
3. searchBooks: 입력 받은 단어 리스트가 제목에 포함된 도서의 ISBN 코드를 반환하는 함수
4. toImageHtml: 이미지 링크를 HTML 형식으로 변환하는 함수
5. showBooks: 입력 받은 ISBN 코드에 대응하는 도서를 보여주고 ISBN 코드를 반환하는 함수
6. checkBook: 입력 받은 ISBN 코드에 대응하는 도서에 호불호를 표시하는 함수

In [2]:
def setPart(word: str):
    consonant = (ord(word[-1]) - 44032) % 28
    if consonant: # 마지막 글자에 받침이 있는 경우
        return '을', '과'
    return '를', '와' # 마지막 글자에 받침이 없는 경우

def setChar(word: str):
    word = word.replace('&nbsp;', ' ')
    word = word.replace('&lt;', '<')
    word = word.replace('&gt;', '>')
    word = word.replace('&amp;', '&')
    word = word.replace('&#035;', '#')
    return word

def searchBooks(words: list):
    search_idx = []
    for i in range(len(data)):
        if all(word in data.iloc[i][2] for word in words):
            search_idx.append(i)
    return list(data.iloc[search_idx]['isbn13'])

def toImageHtml(path: str):
    return '<img src="' + path + '" width="60">'

def showBooks(isbn_list: list, by = [], key = 1):
    target = data_by_isbn.loc[isbn_list].sort_values(by = by)
    if key:
        target.set_index(keys = [list(range(1, len(target) + 1))], drop = True, inplace = True)
    else:
        target.set_index(keys = [['1등', '2등', '3등', '4등', '5등']], drop = True, inplace = True)
    show = target[['img_url', 'title', 'author', 'publisher', 'price', 'pub_date_2']]
    show.columns = ['표지', '제목', '저자', '출판사', '가격', '발행일자']
    format_dict = {}
    format_dict['표지'] = toImageHtml
    display(HTML(show.to_html(escape = False, formatters = format_dict)))
    return list(target.isbn13)

def checkBook(isbn: int):
    title = setChar(data_by_isbn.loc[isbn][2])
    print(f"\n'{title}'{setPart(title)[0]} 선택하셨습니다.")
    showBooks([isbn])
    print('도서 상세 정보: https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=%d'%data_by_isbn.loc[isbn].item_id)
    
    check_list = ['재밌어요', '관심있어요', '별로예요', '그대로 두기']
    overlap = True
    
    if isbn in like:
        print("\n이미 '재밌어요' 표시를 한 도서입니다.")
        del check_list[0]
    elif isbn in interest:
        print("\n이미 '관심있어요' 표시를 한 도서입니다.")
        del check_list[1]
    elif isbn in hate:
        print("\n이미 '별로예요' 표시를 한 도서입니다.")
        del check_list[2]
    else:
        overlap = False
        print()
        del check_list[3]
    
    while True:
        print('해당 도서에 어떤 표시를 하시겠습니까?')
        for i in range(3):
            print(f'{i + 1}. {check_list[i]}')
        
        try:
            check_num = int(input('번호 입력 >>> '))
            check = check_list[check_num - 1]
            
            if check == '그대로 두기':
                print('기존 목록에 그대로 두었습니다.\n')
                break
            
            if overlap:
                if isbn in like:
                    prev = '재밌어요'
                    like.remove(isbn)
                elif isbn in interest:
                    prev = '관심있어요'
                    interest.remove(isbn)
                else:
                    prev = '별로예요'
                    hate.remove(isbn)
            
            if check == '재밌어요':
                like.append(isbn)
            elif check == '관심있어요':
                interest.append(isbn)
            else:
                hate.append(isbn)
            
            if overlap:
                print(f"'{prev}' 목록에서 '{check}' 목록으로 옮겼습니다.")
            else:
                print(f"'{check}' 목록에 추가했습니다.")
            
            if check == '재밌어요':
                reco_idx = recommendIsbn(isbn)
                reco_isbn = list(data.iloc[reco_idx].isbn13)
                print('\n이런 책들은 어떠세요?')
                print(f"재밌다고 표시한 '{title}'{setPart(title)[1]} 유사한 도서 TOP 5 입니다.")
                showBooks(reco_isbn, key = 0)
                
            print()
            break
        
        except:
            print('보기에 없는 입력값입니다. 다시 입력해주세요.\n')

### 각종 기능 함수
목적: 시스템의 주요/부가 기능을 작동시키기 위해
1. openTeum: 시스템을 시작할 때 기능을 선택하는 함수
2. recommendTeum: 도서에 호불호를 표시하고 추천 도서를 확인하는 함수
3. eventTeum: 도서 관련 이벤트를 확인하는 함수
4. userTeum: 사용자의 프로필과 호불호 도서 목록을 확인하는 함수
5. closeTeum: 시스템을 종료하는 함수

In [3]:
def openTeum():
    while True:
        print('\n어떤 기능을 선택하시겠습니까?')
        print('1. 책볼틈 (추천)\n2. 책쉴틈 (이벤트)\n3. 나의틈 (프로필)\n0. 덮을틈 (종료)')
        
        open_select = input('번호 입력 >>> ')
        if open_select in ['1', '2', '3', '0']:
            break
        print('보기에 없는 입력값입니다. 다시 입력해주세요.')
    
    return open_select

In [4]:
def recommendTeum():
    global name
    print('\n[책볼틈]은 도서에 호불호를 표시하고 취향에 맞는 추천 도서를 확인하는 공간입니다.')
    
    while True:
        print('어떤 기능을 선택하시겠습니까?')
        print('1. 도서 검색 및 표시\n2. 추천 도서\n0. 뒤로 가기')
        
        reco_select = input('번호 입력 >>> ')
        if reco_select == '0':
            break
        elif reco_select == '1':
            print('\n검색할 도서의 제목 키워드를 입력해주세요.')
            keywords = input('키워드 입력 >>> ').split()
            search_isbn = searchBooks(keywords)
            if search_isbn:
                print('\n해당 키워드가 포함된 도서 목록입니다.')
                sort_search_isbn = showBooks(search_isbn, by = ['title'])
                while True:
                    print("\n검색할 도서의 번호를 입력해주세요. (뒤로 가기는 '0' 입력)")
                    try:
                        num = int(input('번호 입력 >>> '))
                        if num == 0:
                            print()
                        else:
                            isbn = sort_search_isbn[num - 1]
                        break
                    except:
                        print('보기에 없는 입력값입니다. 다시 입력해주세요.')
                
                if num:
                    checkBook(isbn)
            else:
                print('\n해당 키워드를 포함한 도서가 존재하지 않습니다…ㅠㅠ')
                print('모든 도서를 검색할 수 있도록 계속 노력하는 책틈이 되겠습니다.\n')
        elif reco_select == '2':
            if len(like) == 0:
                print("\n'재밌어요' 도서 목록이 존재하지 않아 추천 도서를 찾을 수 없습니다…")
            elif len(like) == 1:
                title = setChar(data_by_isbn.loc[like[0]][2])
                print(f"\n{name}님의 '재밌어요' 도서인 '{title}'{setPart(title)[0]} 기반으로 추천하는 도서 목록입니다.")
                showBooks(list(data.iloc[recommendIsbn(like[0])].isbn13))
            else:
                print(f"\n{name}님의 '재밌어요' 도서 목록을 기반으로 추천하는 도서 목록입니다.")
                print('(정확한 추천을 위해 시간이 다소 소요될 수 있습니다. 양해 부탁드립니다.)')
                showBooks(likeIsbn())
            print()
        else:
            print('보기에 없는 입력값입니다. 다시 입력해주세요.\n')
    
    return openTeum()

In [5]:
def eventTeum():
    print('\n[책쉴틈]은 도서 관련 이벤트를 확인하는 공간입니다.')
    print('알라딘에서 제공하는 최신 이벤트입니다.')
    
    title, date, link = getEvent()
    for i in range(5):
        print('\n제목 :', setChar(title[i]))
        print(date[i])
        print('링크 :', link[i])
    
    print('\n더 많은 이벤트 보러 가기 >>>', event_url)
    
    return openTeum()

In [6]:
def userTeum():
    global name
    print(f'\n[나의틈]은 {name}님의 프로필과 호불호 도서 목록을 확인하는 공간입니다.')
    
    while True:
        print('어떤 기능을 선택하시겠습니까?')
        print("1. 닉네임 변경\n2. '재밌어요' 도서 목록 확인\n3. '관심있어요' 도서 목록 확인\n4. '별로예요' 도서 목록 확인\n0. 뒤로 가기")
        user_select = input('번호 입력 >>> ')
        
        if user_select == '0':
            break
        elif user_select == '1':
            print(f'\n현재 닉네임: {name}')
            new_name = input('변경 닉네임: ')
            print(f"닉네임을 변경했습니다. '{name}' → '{new_name}'\n")
            name = new_name
        elif user_select in ['2', '3', '4']:
            if user_select == '2':
                check, select_list = '재밌어요', like
            elif user_select == '3':
                check, select_list = '관심있어요', interest
            else:
                check, select_list = '별로예요', hate
            print(f"\n{name}님이 '{check}'라고 표시한 도서 목록입니다.")
            list_isbn = showBooks(select_list, by = ['title'])
            while True:
                print(f"\n삭제하고 싶은 도서의 번호를 입력해주세요. (삭제할 도서가 없으면 '0' 입력)")
                try:
                    del_num = int(input('번호 입력 >>> '))
                    if del_num == 0:
                        print()
                        break
                    del_isbn = list_isbn[del_num - 1]
                    del_title = setChar(data_by_isbn.loc[del_isbn][2])
                    select_list.remove(del_isbn)
                    print(f"'{del_title}'{setPart(del_title)[0]} '{check}' 표시 도서 목록에서 삭제했습니다.\n")
                    break
                except:
                    print('보기에 없는 입력값입니다. 다시 입력해주세요.')
        else:
            print('보기에 없는 입력값입니다. 다시 입력해주세요.\n')
    
    return openTeum()

In [7]:
def closeTeum():
    print('\n[덮을틈]은 책틈을 덮어 종료하는 공간입니다.')
    
    while True:
        print('책틈을 종료하시겠습니까? (y/n)')
        exit_select = input('답변 입력 >>> ').lower()
        
        if exit_select in ['y', 'yes', 'ㅛ', 'n', 'no', 'ㅜ']:
            break
        print('보기에 없는 입력값입니다. 다시 입력해주세요.\n')
    
    if exit_select in ['y', 'yes', 'ㅛ']:
        print('\n<책틈>을 덮습니다. 이용해주셔서 감사합니다.')
        return False
    else:
        return openTeum()

### 시스템 함수
목적: 모든 함수를 총괄하여 사용하기 위해

In [8]:
def BookTeum():
    global name
    
    print('도서 추천 프로그램 <책틈>을 열어주셔서 감사합니다!')
    print('\n당신의 닉네임을 입력해주세요.')
    name = input('이름 입력 >>> ')
    
    print(f'\n환영합니다, {name}님.')
    
    select = openTeum()
    while True:
        if select == '1':
            select = recommendTeum()
        elif select == '2':
            select = eventTeum()
        elif select == '3':
            select = userTeum()
        elif select == '0':
            select = closeTeum()
        else:
            break